## Notebook to get weather data

The purpose of this notebook is to create a dataframe which has weather data for given venue in given date. This is done in three major parts:

1. Create geolocations and get longitude and latitude data for given baseball venue
2. Use VisualCrossing API to get weather data for all matches between the 2017 and 2018 seasion in given location in given day. 
3. Merge and clean these dataframes so that they can be used for further analysis

In [2]:
import pandas as pd 

import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import requests as re 

import json

from datetime import datetime

pd.set_option('display.max_columns', None)

### 1) Getting geolocations based on venue

We load up a preprocessed dataframe, and get all locational data with the use of geopy

In [2]:
# load data from disk

df = pd.read_csv('Baseball_Merged.csv')

In [3]:
# We initialise a geocoder object using the combination of RateLimiter and Nominatim (both are from the geopy package)
# these will be used to get geolocations based on data that we have, which is the name of the baseball pitch

geocoder = RateLimiter(Nominatim(user_agent='ba').geocode, min_delay_seconds=1)

# We will get the unique venues so that we can apply geocoder to those
venues = df['venue_name'].unique()

location_dict = {}

for i in venues: 
    location_dict[i] = geocoder(i)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Wrigley Field',), **{}).
Traceback (most recent call last):
  File "C:\Users\T450s\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 421, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\T450s\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 416, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\T450s\Anaconda3\lib\http\client.py", line 1344, in getresponse
    response.begin()
  File "C:\Users\T450s\Anaconda3\lib\http\client.py", line 306, in begin
    version, status, reason = self._read_status()
  File "C:\Users\T450s\Anaconda3\lib\http\client.py", line 267, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\T450s\Anaconda3\lib\socket.py", line 589, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\T450s\Anaconda3\lib\site-packages\urllib3\contrib\pyopenssl.py

In [4]:
# Geocoder is very powerful but it is not working 100% given that we only feed in the name of a baseball pitch
    # some of which might not even exist anymore

# We will identify potentially erroneous locations by looking at the type of the observation - if this is not a stadium
# we do a manual inspection 
    
error_pairs = {}
for venue in venues:
    try:
        if location_dict[venue].raw['type'] != 'stadium':
            error_pairs[venue] = location_dict[venue].raw['type']
    except:
        error_pairs[venue] = 'None'

# Searching on the net we found the correct names for stadiums that were erroneously mapped.
# We will do a new geolocation search for the new names later


rename_dict = {'Miller Park':'American Family Field',
'O.co Coliseum': 'RingCentral Coliseum',
'U.S. Cellular Field' : 'Guaranteed Rate Field',
'AT&T Park' : 'Oracle Park',
'Oakland Coliseum' : 'RingCentral Coliseum',
'Fort Bragg Field' : 'Fort Bragg Field baseball',
'Williamsport Little League Classic' : 'Williamsport, Pennsylvania',
'Angel Stadium': 'Angel Stadium of Anaheim',
'BB&T Ballpark': 'Williamsport, Pennsylvania'
}

In [5]:
# We will take the new dictionary, which is rename dict, and find every geolocatoin that correspond to the new names
# after that we append these new names to the already existing location_dict

for k,v in rename_dict.items():
    location_dict[v] = geocoder(v)
    
# After that we remove the previously found, but erroneous names

for key in [k for k,v in rename_dict.items()]:
    if key in location_dict:
        del location_dict[key]

RateLimiter caught an error, retrying (0/2 tries). Called with (*('American Family Field',), **{}).
Traceback (most recent call last):
  File "C:\Users\T450s\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 421, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\T450s\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 416, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\T450s\Anaconda3\lib\http\client.py", line 1344, in getresponse
    response.begin()
  File "C:\Users\T450s\Anaconda3\lib\http\client.py", line 306, in begin
    version, status, reason = self._read_status()
  File "C:\Users\T450s\Anaconda3\lib\http\client.py", line 267, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\T450s\Anaconda3\lib\socket.py", line 589, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\T450s\Anaconda3\lib\site-packages\urllib3\contrib\pyop

In [80]:
# We create subdf, which is simple table that has unique combinations of dates and venues
subdf = df[['date','venue_name']].drop_duplicates()

# We map the new venue name to the dataframe we created earlier
to_replace = [k for k,v in rename_dict.items()]

new_venue = []

for i in subdf['venue_name']:
    if i in to_replace:
        new_venue.append(rename_dict[i])
    else:
        new_venue.append(i)

subdf['new_venue'] = new_venue

# Now we add locational data and longitudes and latitudes
subdf['location'] = [location_dict[i] for i in subdf['new_venue']]
subdf['latitude'] = [i.latitude for i in subdf['location']]
subdf['longitude'] = [i.longitude for i in subdf['location']]

In [135]:
# Writing resulting df to disk

subdf.to_csv('date_n_location.csv')

### 2) Get weather data with API 

We will use the Visualcrossing API to get weather data for given location on given day

In [4]:
# Load data from disk
subdf= pd.read_csv('date_n_location.csv', index_col = 'Unnamed: 0')

# We will filter out everything that did not happen in the past two years. For this we first need to convert 
# the date variable to have date datatype using the datetime package

subdf['date'] = subdf.apply(lambda x : datetime.strptime(x['date'], "%Y-%m-%d").date(),axis= 1)
subdf = subdf[subdf['date'] > datetime.strptime('2017-04-01','%Y-%m-%d').date()]

In [6]:
# This is commented out so that there is no chance I rerun it
# This cell is to read in the API key stored in a separate txt file on my PC

#f = open(r"C:\Users\T450s\Desktop\api\weather/visualcrossing_weather.txt", "r")
#api_key = f.readline()
#f.close()

In [5]:
# Function to create API link

# We are using VisualCrossing Weather Data Services in order to query historic weather data.
# We pay 0.0001$/record, so we will be extra careful when requesting data. 
# Documentation for the API is available from here: 
    # https://www.visualcrossing.com/resources/documentation/weather-data/getting-started-with-weather-data-services/

# We will be using daily data and request the info to come to us as a json

def keygen(latitude,longitude,hours,start_date,end_date,json_or_csv,api_key):
    
    BaseURL = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/'
    hist_forecast = 'history/'
    
    
    locations = '?locations='+ str(latitude) + ',' + str(longitude)
    aggregateHours = 'aggregateHours=' + str(hours)
    startDateTime = 'startDateTime=' + str(start_date) + 'T00:00:00' 
    endDateTime = 'endDateTime=' + str(end_date) + 'T00:00:00'
    unitGroup = 'unitGroup='+'uk'
    contentType = 'contentType='+json_or_csv
    dayStartTime = 'dayStartTime='+'0:0:00'
    dayEndTime = 'dayEndTime='+'0:0:00'
    key = 'key=' + str(api_key)
    
    
    res = '&'.join([locations,aggregateHours,startDateTime,endDateTime,unitGroup,contentType,dayStartTime,dayEndTime,key])
    request_link = BaseURL+hist_forecast + res
    return(request_link)

In [7]:
# I will drop a few variables because they are events happening with very low frequnency 
# (maybe some temporary fields or special competitions) - it is not worth including these in the analysis

to_drop = ['Estadio de Beisbol Monterrey', 'Fort Bragg Field baseball', 'Hiram Bithorn Stadium', 'Williamsport, Pennsylvania']
subdf = subdf[~subdf['new_venue'].isin(to_drop)]

In [10]:
# This cell is to create an API dataframe the purpose of which is to provide a list of links that can be used for the API
# We need a start date, end date, latitude, longitud information to create API links


# create a column with the start date
start_df = subdf.groupby(['longitude', 'latitude']).agg({'date':'min'}).rename(columns = {'date':'start_date'}).reset_index()

# create a column with end date
end_df = subdf.groupby(['longitude', 'latitude']).agg({'date':'max'}).rename(columns = {'date':'end_date'}).reset_index()

# merge dfs 
api_df = pd.merge(start_df,end_df)

# create links with custom function specified above
api_df['links'] = api_df.apply(lambda x: keygen(x['latitude'],x['longitude'],24,x['start_date'],x['end_date'],'json',api_key) ,axis = 1)

In [75]:
# create empty list to append df to cache dfs resulting from interation
weather_df = []

for i, c in api_df.iterrows():
    
    # We first get a json from API and save it to myjson variable
    myjson = re.get(api_df.loc[i,'links'])
    
    # weather_json will hold the parsed json that is now a dictionary 
    weather_json = json.loads(myjson.text)
    
    # get coordinates as json's structure needs such a key
    my_coords =  [str(k) for k in [weather_json['locations'].keys()][0]][0]
    
    # printing to see status only
    print(str(api_df.loc[i,'latitude']) + ',' + str(api_df.loc[i,'longitude']), 'is ready')
    
    # create temporary dataframe out of json
    temp_weather_df = pd.DataFrame(weather_json['locations'][my_coords]['values'])
    
    # add location var so that it can be joined with sub df 
    temp_weather_df['location'] = my_coords

    # append to list created in the beginning to concatenate them 
    weather_df.append(temp_weather_df)

# concatenate list of dfs into one final df 
weather_df = pd.concat(weather_df)

34.074552350000005,-118.24004805779221
33.8002599,-117.88174262759796
32.70718815,-117.15687745290563
33.44548555,-112.06669283293144
39.756031400000005,-104.99292855531492
32.7513904,-97.08347649984135
29.75723205,-95.35523391491142
39.05144525,-94.48047131020273
44.9817039,-93.2778457595517
38.62255379999999,-90.19392193458769
43.0280619,-87.97209586840819
41.9481225,-87.6563513145702
41.8300162,-87.63256264837347
39.09724845,-84.50662325533993
33.89070855,-84.46853422885837
42.33915895,-83.04962481782741
27.7680559,-82.65327550461797
41.4960144,-81.68420022215649
25.7782474,-80.21980500744203
40.446925799999995,-80.00560626612204
43.6416641,-79.38919882366382
38.87274095,-77.00838588569519
39.28398230000001,-76.62249149865416
39.90588575,-75.16541101747245
40.82958275,-73.92652118491901
40.75727785,-73.84587884942417
42.346462100000004,-71.0971002033302


In [88]:
# Write weather data to disk so that we don't have to rerun API requests - this would cost money.

weather_df.to_csv('weather_df.csv')

### 3) Merge resulting tables

In [89]:
# load up dataframes from disk

weather_df = pd.read_csv('weather_df.csv',index_col = 'Unnamed: 0')
subdf= pd.read_csv('date_n_location.csv', index_col = 'Unnamed: 0')

In [81]:
# some minor formatting to be used for joining 

subdf['latlong'] = subdf['latitude'].astype('str') +',' + subdf['longitude'].astype('str')
weather_df['date'] = [datetime.fromtimestamp(date/1000).strftime('%Y-%m-%d') for date in weather_df['datetime']]

In [82]:
# merge the datasets so now we will have date-location-weather data in the same dataframe

merged_df = pd.merge(left = subdf, 
         right = weather_df,
         left_on = ['date', 'latlong'],
         right_on = ['date','location' ], how = 'inner')

In [85]:
# clean this dataframe so that it only has columns that are useful

merged_df = merged_df[['date', 'venue_name', 'latitude','longitude',
                       'wdir', 'temp', 'maxt', 'visibility', 'wspd', 
                       'cloudcover', 'mint', 'precip', 'snowdepth', 
                      'dew', 'humidity','precipcover']]

In [86]:
merged_df.head()

,date,venue_name,latitude,longitude,wdir,temp,maxt,visibility,wspd,cloudcover,mint,precip,snowdepth,dew,humidity,precipcover
0,2017-04-02,Busch Stadium,38.622554,-90.193922,116.71,13.3,19.4,9.9,12.5,11.0,7.3,0.25,0.0,7.0,66.63,4.17
1,2017-04-02,Tropicana Field,27.768056,-82.653276,135.58,24.3,29.4,9.9,11.1,5.5,21.0,0.00,0.0,18.9,73.06,0.00
2,2017-04-02,Chase Field,33.445486,-112.066693,161.79,20.0,27.1,9.9,12.7,22.6,12.1,0.00,0.0,0.6,29.90,0.00
3,2017-04-03,Oakland Coliseum,37.751675,-122.199371,200.42,14.9,20.6,9.5,17.1,17.0,8.9,0.00,0.0,7.5,63.81,0.00
4,2017-04-03,Citi Field,40.757278,-73.845879,194.04,12.9,16.6,9.9,17.0,65.7,8.8,0.00,0.0,0.2,42.47,0.00


In [90]:
merged_df.to_csv('location_weather.csv')